<a href="https://colab.research.google.com/github/tibayo1/Leukemia-DL-Diagnosis/blob/master/leukemia_dl_diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>